In [9]:
%run load_actiwatch_data.py
%run firsttime.py

import matplotlib.pyplot as plt
%matplotlib inline

import pyarrow

from joblib import *

import statsmodels.api as sm
import statsmodels.formula.api as smf

import scipy.stats as stats

# this is used to make Federal Holidays a nonschool day.  Note that we don't have any
# way to recognize school district unique holidays, like teacher work days of such 
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar


In [10]:
# change the values in this cell to process the data desired

global ddirs

# dictionary links a unique experimental group with the directory that holds those files
# inside that directory are unique files per subject in the group
ddirs = {
        'Fall 220 2015': '../Fall 220 2015/',
        'Fall 220 2016': '../Fall 220 2016/',
        'Spring 418 2016': '../Spring 418 2016/',
        'Spring 418 2017': '../Spring 418 2017/',
        'Spring 418 2018': '../Spring 418 2018/',
        'Summer 220 2016': '../Summer 220 2016/',
        'Summer 220 2017': '../Summer 220 2017/',
        'Winter 220 2018': '../Winter 220 2018/',
}

# where the saved processed data goes
outfile = '../processed data/SeaUgrad'

In [11]:
recalculate_raw = False # true forces long recalculations, false loads processed data from disk
recalculate_timing = False

threshs = [ [5], [10], [50], [100], [500], [1000] ] 

def get_raw_data(season):
    rawd, summaryd = load_actiwatch_data(ddirs[season],uidprefix=season)
    rawd['Group']=season
    return rawd    

if recalculate_raw:
    print("Loading raw from disk ...")    
    results = Parallel(n_jobs=len(ddirs))(delayed(get_raw_data)(season) for season in ddirs.keys())
    allData = pd.concat(results)
    # the following assignments depend on the datafile names and directory structure being exactly of the form:
    # ../Quarter Class 4digitYear/uniqueSubjectID_blahblahwhatever.csv
    allData['Quarter']=allData.UID.apply(lambda x: x.split()[0])
    allData['Class']=allData.UID.apply(lambda x: x.split()[1])
    allData['Year']=allData.UID.apply(lambda x: x.split()[2][:4])
    allData['Subject ID']=allData.UID.apply(lambda x: x.split()[2][4:])
    allData.to_parquet(outfile+'raw.parquet',engine='fastparquet',compression='gzip')
else:
    allData = pd.read_parquet(outfile+'raw.parquet')
    
if recalculate_timing:    
    print("Calculating light timing data ...")
    results = Parallel(n_jobs=len(threshs))(delayed(firstAndLastLight)(allData, threshold) for threshold in threshs)
    timingData = pd.concat(results)
    print("Adding holiday markers to timing data ...")
    cal = calendar()
    holidays = cal.holidays(start=timingData.Date.min(), end=timingData.Date.max())

    nn = pd.DatetimeIndex( timingData.Date )
    timingData['DayofWeek'] = nn.dayofweek 
    days = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
    daytype = ['Weekday','Weekday','Weekday','Weekday','Weekday','Weekend/Holiday','Weekend/Holiday']

    daygrp=[]
    dtpgrp=[]
    wkendholiday=[]
    for k, row in timingData.iterrows():
        daygrp.append(row['Group'].split('seattle')[0] + days[row['DayofWeek']])
        if holidays.isin([row['Date']]).any():
            dtpgrp.append(row['Group'].split('seattle')[0] + 'Weekend/Holiday')
            wkendholiday.append(True)
        else:
            dtpgrp.append(row['Group'].split('seattle')[0] + daytype[row['DayofWeek']])
            if row['DayofWeek'] > 4:
                wkendholiday.append(True)
            else:
                wkendholiday.append(False)

    timingData['GroupDayofWeek'] = daygrp
    timingData['GroupDayType'] = dtpgrp
    timingData['Weekend/Holiday'] = wkendholiday
    
    alluids = timingData.UID.unique()

    awake_rows = {}
    for uid in alluids:
        awake_rows[uid] = (allData.UID == uid) & (allData['Interval Status'].isin(['ACTIVE','REST'])) & np.logical_not(allData['Off-Wrist Status'])

    totalact=[]
    tabvlight=[]

    for row in range(len(timingData)):
        if row%100 == 0:
            print(row, ' / ', len(timingData))

        entry = timingData.iloc[row,:]
        #day = entry.iloc[0].date()
        day = entry.Date
        thisday = pd.to_datetime(day).isoformat()
        uid = entry.UID

        daylight = allData['White Light'][awake_rows[uid]][thisday + ' 00:00:00' : thisday + ' 23:59:59'] 
        abvthresh = daylight[(daylight>entry['Threshold'])]
        if len(daylight)>15:
            dperiod = pd.Timedelta(np.min(daylight.index[1:14]-daylight.index[0:13]))
            tabvlight.append( dperiod * len(abvthresh))
        else:
            tabvlight.append( pd.Timedelta('0 minutes') )


        totalact.append(allData['Activity'][awake_rows[uid]][thisday + ' 00:00:00' : thisday + ' 23:59:59'].sum())

    timingData['totalactivity']=totalact
    timingData['timeabvluxlevel']=[ el.total_seconds()/60.0 for el in tabvlight]
    
    timingData['Quarter']=timingData.UID.apply(lambda x: x.split()[0])
    timingData['Class']=timingData.UID.apply(lambda x: x.split()[1])
    timingData['Year']=timingData.UID.apply(lambda x: x.split()[2][:4])
    timingData['Subject ID']=timingData.UID.apply(lambda x: x.split()[2][4:])

    #timingData.to_parquet(outfile+'timing.parquet')
    # at this moment there are two choices for parquet writing, pyarrow (which does not support Timedelta)
    # and fastparquet (which does not support datetime.date )... I'm going to go with the latter
    tdc = timingData.copy()
    tdc.Date = tdc.Date.astype('datetime64')
    tdc['Watch period'] = pd.to_timedelta(tdc['Watch period']) #oops that was a datetime.timedelta, which we dont need that also choked parquet
    tdc.to_parquet(outfile+'timing.parquet', engine='fastparquet', compression='gzip')
    del(tdc)
else:
    tdc = pd.read_parquet(outfile+'timing.parquet', engine='fastparquet')
    tdc.Date = tdc.Date.apply(lambda x: x.date()) # go back to original data format
    timingData = tdc.copy()

    

In [12]:
touse = pd.read_excel('../Subjects Included in Study.xlsx')
checkme = touse[['Quarter','Year','Class','Subject ID']].drop_duplicates().sort_values(by=['Quarter', 'Year','Class','Subject ID'])
print(checkme.dtypes)
checkme

Quarter       object
Year           int64
Class          int64
Subject ID     int64
dtype: object


,Quarter,Year,Class,Subject ID
0,Fall,2015,220,1
1,Fall,2015,220,2
2,Fall,2015,220,3
3,Fall,2015,220,5
4,Fall,2015,220,6
...,...,...,...,...
438,Winter,2018,220,116
439,Winter,2018,220,117
440,Winter,2018,220,118
441,Winter,2018,220,119


In [13]:
dataids = timingData[['Quarter','Year','Class','Subject ID']].drop_duplicates()
dataids['Subject ID'] = dataids['Subject ID'].astype(int)
dataids['Year'] = dataids['Year'].astype(int)
dataids['Class'] = dataids['Class'].astype(int)
dataids.sort_values(by=['Quarter', 'Year','Class','Subject ID'],inplace=True)
dataids.reset_index(drop=True,inplace=True)
print(dataids.dtypes)
dataids

Quarter       object
Year           int64
Class          int64
Subject ID     int64
dtype: object


,Quarter,Year,Class,Subject ID
0,Fall,2015,220,1
1,Fall,2015,220,2
2,Fall,2015,220,3
3,Fall,2015,220,5
4,Fall,2015,220,6
...,...,...,...,...
504,Winter,2018,220,116
505,Winter,2018,220,117
506,Winter,2018,220,118
507,Winter,2018,220,119


In [14]:
df_all = pd.concat( [dataids, checkme])
rmall = df_all.drop_duplicates(keep=False, inplace=False)
rmall['UID']=rmall.apply(axis=1, func=lambda x: '{} {} {}{:04d}'.format(x['Quarter'],x['Class'],x['Year'],x['Subject ID']))
rmall

/Users/jfleischer/opt/anaconda3/envs/light_analysis_2020/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Quarter,Year,Class,Subject ID,UID
463,Winter,2018,220,57,Winter 220 20180057
503,Winter,2018,220,115,Winter 220 20180115


In [15]:
rmweekends = touse.loc[ touse.Notes.dropna().index, ['Quarter','Year','Class','Subject ID'] ]
rmweekends['UID']=rmweekends.apply(axis=1, func=lambda x: '{} {} {}{:04d}'.format(x['Quarter'],x['Class'],x['Year'],x['Subject ID']))
rmweekends

,Quarter,Year,Class,Subject ID,UID
67,Fall,2015,220,79,Fall 220 20150079
93,Fall,2015,220,114,Fall 220 20150114
200,Summer,2016,220,49,Summer 220 20160049
216,Summer,2016,220,65,Summer 220 20160065
229,Summer,2016,220,80,Summer 220 20160080
232,Fall,2016,220,1,Fall 220 20160001
244,Fall,2016,220,17,Fall 220 20160017
255,Fall,2016,220,30,Fall 220 20160030
323,Spring,2017,418,62,Spring 418 20170062
397,Winter,2018,220,55,Winter 220 20180055


In [16]:
#remove subjects who don't belong
thedata = allData.copy()
thetiming = timingData.copy()
print(thedata.shape, thetiming.shape)
for anid in rmall.UID:
    thedata = thedata.query('~(UID == @anid)')
    thetiming = thetiming.query('~(UID == @anid)')
print(thedata.shape,  thetiming.shape)

(43664740, 15) (47844, 23)
(43544129, 15) (47712, 23)


In [17]:
thedata['is_weekend'] = thedata.index.dayofweek.isin([5,6])
thedata['is_weekend'].describe()

count     43544129
unique           2
top          False
freq      31506511
Name: is_weekend, dtype: object

In [18]:
# ratio of weekday to total datapoints is close ot the expected values
print(31.5/43.5, 5./7.)

0.7241379310344828 0.7142857142857143


In [19]:
thetiming['OutofSchool'] = thetiming['Weekend/Holiday']

In [20]:
# remove weekend dates for subjects who shouldn't be in the weekend data
print(thedata.shape, thetiming.shape)
for anid in rmweekends.UID:
    thedata = thedata.query('~((UID == @anid)&(is_weekend == True))')
    thetiming = thetiming.query('~((UID == @anid)&(OutofSchool == True))')
print(thedata.shape, thetiming.shape)

(43544129, 16) (47712, 24)
(43244609, 16) (47430, 24)


In [21]:
thedata['is_weekend'].describe()

count     43244609
unique           2
top          False
freq      31506511
Name: is_weekend, dtype: object

In [22]:
# ratio is still good
31.5/43.2

0.7291666666666666

In [23]:
# OK calculate on only the good data
allData=thedata

In [24]:
allddate= pd.Series( allData.index.date )
holidays = calendar().holidays(start=timingData.Date.min(), end=timingData.Date.max())
allData['is_holiday'] = allddate.apply( lambda x: holidays.isin([x]).any())
allData['OutofSchool'] = allData['is_weekend'] | allData['is_holiday']

In [25]:
allData['Group'].unique()

array(['Fall 220 2015', 'Fall 220 2016', 'Spring 418 2016',
       'Spring 418 2017', 'Spring 418 2018', 'Summer 220 2016',
       'Summer 220 2017', 'Winter 220 2018'], dtype=object)

In [26]:
allData['Group'].unique()

array(['Fall 220 2015', 'Fall 220 2016', 'Spring 418 2016',
       'Spring 418 2017', 'Spring 418 2018', 'Summer 220 2016',
       'Summer 220 2017', 'Winter 220 2018'], dtype=object)

In [27]:
dpart = allData['OutofSchool'].apply(lambda x: 'Non-school day' if x else 'School day')
allData['GroupDayType'] = allData['Group'].str.cat( dpart, sep=' ')
allData['UIDDayType'] = allData['UID'].str.cat( dpart, sep=' ')

In [28]:
def hours_float( dttm ):
    # takes timestamp, returns floating point hours description of the time  
    if pd.isna(dttm):
        return np.NaN
    
    td = pd.Timedelta( dttm.time().isoformat() )
    return td.total_seconds() / 3600. 

hours_float( thetiming['First Light'].iloc[0] )

5.920833333333333

In [29]:
thetiming['firstlight']=thetiming['First Light'].apply(lambda x: hours_float(x))
thetiming['lastlight']=thetiming['Last Light'].apply(lambda x: hours_float(x))
thetiming['lastlight']=thetiming['lastlight'].apply( lambda x: x if x>4.0 else x+24.)

In [30]:
from astral import *
a = Astral()

dawns=[]
dusks=[]
dlookup = []
dates = thetiming.Date.unique()

for day in dates:
    dlookup.append( a['Seattle'].sun(date=day) )
    
lookupday = pd.Series(dlookup,index=dates)
 
print('Sunset calc')
thetiming['Sunset'] = thetiming.Date.apply( lambda x: hours_float( lookupday[x]['sunset'] ) )
print('Sunrise calc')
thetiming['Sunrise'] = thetiming.Date.apply( lambda x: hours_float( lookupday[x]['sunrise'] ) )

Sunset calc
Sunrise calc


In [31]:
thetiming['hours from sunrise to first light abv threshold'] = thetiming['firstlight'] - thetiming['Sunrise']
thetiming['hours from sunset to last light abv threshold'] = thetiming['lastlight'] - thetiming['Sunset']

In [32]:
thetiming.GroupDayType = thetiming.GroupDayType.apply(lambda x: x.replace('Weekday',' School day').replace('Weekend/Holiday',' Non-school day'))

In [33]:
thetiming.GroupDayType = thetiming.GroupDayType.apply(lambda x: x.replace(' School day','\nSchool day').replace(' Non-school day','\nNon-school day'))

In [38]:
thetiming.Date = thetiming.Date.astype('datetime64')

In [39]:
# now lets add in the sleep info to the timing data per person day
sleepons = []
sleepoffs = []
sleepdurs = []
sleep2durs = []
sleeponMSLM = []
sleepoffMSLM = []

for arow in thetiming.itertuples():    
    UID = arow.UID
    DT = arow.Date
    TM = DT+pd.Timedelta('1 day')
    today = DT.strftime('%Y-%m-%d')
    nextday = TM.strftime('%Y-%m-%d')

    # raw data for this timingData entry, 
    # split sleep day at 6PM because I assume nobody is asleep at this time
    xx = allData.query("UID == @UID").loc[today+' 18:00':nextday+' 18:00']
    asleep = xx[ xx['Interval Status']=='REST-S'].copy()
    
    # if no asleep moments in this day; maybe watch started today so it records light,
    # but was removed from wrist before sleep so EXCLUDED
    if asleep.shape[0] == 0: 
        sleepons.append(pd.NaT)
        sleepoffs.append(pd.NaT)
        sleepdurs.append(pd.NaT)
        sleep2durs.append(pd.NaT)
        sleeponMSLM.append(pd.np.NaN)
        sleepoffMSLM.append(pd.np.NaN)
        continue
        
    # there might be multiple sleep periods in a given days data, find them
    # assume that any time there is more than 1 hr between sucessive REST-S entries
    # it constitutes a new sleep period
    sleep_period = []
    per = 0
    lt = asleep.index[0]
    for t in asleep.index:
        if (t-lt > pd.Timedelta('1 hour')): # allow up to 1 hour awake in the middle of the night 
            per = per + 1
        lt = t
        sleep_period.append(per)
    asleep['Sleep period']=sleep_period
    
    # calc sleep onsets/offsets/duration for each period of sleep in a person-day of data
    sleeps = asleep.reset_index().groupby('Sleep period').apply( lambda x: pd.DataFrame({
        'Sleep onset':x.DateTime.min(), 'Sleep offset':x.DateTime.max(), 'Sleep duration':x.DateTime.max()-x.DateTime.min()}, index = x.DateTime.dt.normalize() ) )
    sleeps = sleeps.drop_duplicates().sort_values(by='Sleep duration',ascending=False)
    nsleeps = sleeps.shape[0]
    if nsleeps>1:
        secondary_sleep_dur = sleeps['Sleep duration'].iloc[1:].sum()
    else:
        secondary_sleep_dur = pd.Timedelta('0')
    onset = sleeps.iloc[0]['Sleep onset']
    offset = sleeps.iloc[0]['Sleep offset']
    dur =  sleeps.iloc[0]['Sleep duration']
    onMSLM = (onset - DT).total_seconds()/60.
    offMSLM = (offset - TM).total_seconds()/60.
    sleepons.append(onset)
    sleepoffs.append(offset)
    sleepdurs.append(dur)
    sleep2durs.append(secondary_sleep_dur)
    sleeponMSLM.append(onMSLM)
    sleepoffMSLM.append(offMSLM)

thetiming['Sleep onset'] = sleepons
thetiming['Sleep offset'] = sleepoffs
thetiming['Sleep duration'] = sleepdurs
thetiming['Secondary sleep duration'] = sleep2durs
thetiming['Sleep onset MSLM'] = sleeponMSLM
thetiming['Sleep offset MSLM'] = sleepoffMSLM

In [116]:
# it seems its possible some manipulations I did earlier screwed up the datatype... force it
thetiming['Sleep onset'] = thetiming['Sleep onset'].apply( lambda x: pd.to_datetime(x))
thetiming['Sleep offset'] = thetiming['Sleep offset'].apply( lambda x: pd.to_datetime(x))

thetiming['Sleep duration'] = thetiming['Sleep duration'].apply( lambda x: pd.Timedelta(x))
thetiming['Secondary sleep duration'] = thetiming['Secondary sleep duration'].apply( lambda x: pd.Timedelta(x))

In [117]:
# there was some accidental duplication the first time I was working out this analysis, won't be necessary in the future as I fixed the problem
thetiming = thetiming.loc[:,~thetiming.columns.duplicated() ]
thetiming['Watch period'] = pd.to_timedelta(thetiming['Watch period'])
thetiming.to_parquet(outfile+'TimingAnalysis.parquet', engine='fastparquet', compression='gzip')

In [ ]:
result1 = thetiming.query('Threshold == 50').groupby(['UID','OutofSchool'])['firstlight'].describe()
result1.unstack().to_excel('../processed data/Time to 1st 50lux per person.xlsx')

In [56]:
result2 = thetiming.query('Threshold == 50').groupby(['UID','OutofSchool'])['lastlight'].describe()
result2.unstack().to_excel('../processed data/Time to last 50lux per person.xlsx')

In [140]:
result3 = thetiming.query('Threshold == 50')[['UID','Date','First Light','Last Light','Sunrise','Sunset','hours from sunrise to first light abv threshold','hours from sunset to last light abv threshold']]
result3['First light abv threshold (hrs)'] = (result3['First Light'] - result3['Date']).apply( lambda x: x.total_seconds()/3600.0)
result3['Last light abv threshold (hrs)'] = (result3['Last Light'] - result3['Date']).apply( lambda x: x.total_seconds()/3600.0)
result3.to_excel('../processed data/Raw person day 50lux clock times and relative to sun.xlsx')

In [142]:
thetiming[['UID','Date']].drop_duplicates().shape, thetiming.UID.unique().shape

((7905, 2), (507,))

A question: how many person-days have a secondary sleep, and what do they look like?


In [124]:
moresleep = (thetiming['Secondary sleep duration']>pd.Timedelta('0'))
extrasleep = thetiming.loc[moresleep,['Sleep onset','Sleep offset','Sleep duration','Secondary sleep duration']].drop_duplicates()
print(extrasleep.shape)
extrasleep

(33, 4)


,Sleep onset,Sleep offset,Sleep duration,Secondary sleep duration
index,,,,
594,2015-10-10 11:17:00,2015-10-10 14:41:00,03:24:00,02:05:30
1191,2015-10-11 10:13:30,2015-10-11 14:49:15,04:35:45,02:39:00
1318,2015-10-28 21:29:30,2015-10-29 06:42:00,09:12:30,03:08:00
1320,2015-10-26 22:42:45,2015-10-27 06:26:30,07:43:45,01:53:30
2302,2016-04-02 00:15:30,2016-04-02 05:14:30,04:59:00,04:17:00
2902,2016-04-16 01:20:30,2016-04-16 06:46:00,05:25:30,00:13:00
3174,2017-04-10 01:37:15,2017-04-10 07:20:30,05:43:15,01:39:30
3728,2017-03-30 21:25:00,2017-03-31 02:07:45,04:42:45,03:09:30
3729,2017-04-04 01:50:45,2017-04-04 07:29:15,05:38:30,01:23:45


In [125]:
extrasleep['Secondary sleep duration'].describe()

count                        33
mean     0 days 02:10:26.363636
std      0 days 01:34:58.487501
min             0 days 00:05:00
25%             0 days 01:04:45
50%             0 days 01:50:00
75%             0 days 03:09:30
max             0 days 06:48:00
Name: Secondary sleep duration, dtype: object

In [128]:
extrasleep['Sleep duration'].describe()

count                        33
mean     0 days 06:20:20.454545
std      0 days 02:25:01.662140
min             0 days 03:24:00
25%             0 days 04:35:45
50%             0 days 05:36:30
75%             0 days 07:39:15
max             0 days 12:26:45
Name: Sleep duration, dtype: object

In [127]:
# anyone sleep more than two periods?
nsls = []
sleepers = []

for arow in thetiming[moresleep].itertuples():    
    UID = arow.UID
    DT = arow.Date
    TM = DT+pd.Timedelta('1 day')
    today = DT.strftime('%Y-%m-%d')
    nextday = TM.strftime('%Y-%m-%d')

    # raw data for this timingData entry, 
    # split sleep day at 6PM because I assume nobody is asleep at this time
    xx = allData.query("UID == @UID").loc[today+' 18:00':nextday+' 18:00']
    asleep = xx[ xx['Interval Status']=='REST-S'].copy()
    
        
    # there might be multiple sleep periods in a given days data, find them
    # assume that any time there is more than 1 hr between sucessive REST-S entries
    # it constitutes a new sleep period
    sleep_period = []
    per = 0
    lt = asleep.index[0]
    for t in asleep.index:
        if (t-lt > pd.Timedelta('1 hour')): # allow up to 1 hour awake in the middle of the night 
            per = per + 1
        lt = t
        sleep_period.append(per)
    asleep['Sleep period']=sleep_period
    
    # calc sleep onsets/offsets/duration for each period of sleep in a person-day of data
    sleeps = asleep.reset_index().groupby('Sleep period').apply( lambda x: pd.DataFrame({
        'Sleep onset':x.DateTime.min(), 'Sleep offset':x.DateTime.max(), 'Sleep duration':x.DateTime.max()-x.DateTime.min()}, index = x.DateTime.dt.normalize() ) )
    sleeps = sleeps.drop_duplicates().sort_values(by='Sleep duration',ascending=False)
    nsleeps = sleeps.shape[0]
    
    
    if nsleeps>2:
        sleeps['UID']=UID
        sleeps['DT']=DT
        sleeps.reset_index(drop=True).set_index(['UID','DT'])
        sleepers.append(sleeps)

pd.concat(sleepers).reset_index().drop('DateTime',axis=1).set_index(['UID','DT']).drop_duplicates()

Sleep period         Sleep onset  \
UID                 DT                                             
Spring 418 20170038 2017-03-28             1 2017-03-29 04:16:15   
                    2017-03-28             2 2017-03-29 11:19:00   
                    2017-03-28             0 2017-03-28 18:00:00   
Summer 220 20160020 2016-07-06             1 2016-07-07 04:06:45   
                    2016-07-06             2 2016-07-07 13:45:15   
                    2016-07-06             0 2016-07-06 18:00:00   

                                      Sleep offset Sleep duration  
UID                 DT                                             
Spring 418 20170038 2017-03-28 2017-03-29 08:03:30       03:47:15  
                    2017-03-28 2017-03-29 14:52:15       03:33:15  
                    2017-03-28 2017-03-28 18:37:15       00:37:15  
Summer 220 20160020 2016-07-06 2016-07-07 10:55:15       06:48:30  
                    2016-07-06 2016-07-07 18:00:45       04:15:30  
                    2016-07-06 2016-07-06 20:32:30       02:32:30

In [144]:
thetiming.iloc[:10,-10:]

,Sunset,Sunrise,hours from sunrise to first light abv threshold,hours from sunset to last light abv threshold,Sleep onset,Sleep offset,Sleep duration,Secondary sleep duration,Sleep onset MSLM,Sleep offset MSLM
index,,,,,,,,,,
0,18.630000,7.271389,-1.350556,4.953333,2015-10-09 00:01:30,2015-10-09 05:43:00,05:41:30,0 days,1441.50,343.00
1,18.531667,7.341667,-0.479167,4.780833,2015-10-11 23:47:45,2015-10-12 05:59:30,06:11:45,0 days,1427.75,359.50
2,18.466944,7.389167,0.969167,4.166389,2015-10-14 00:17:45,2015-10-14 05:43:30,05:25:45,0 days,1457.75,343.50
3,18.403333,7.436944,-1.166111,4.567500,2015-10-16 00:07:00,2015-10-16 04:41:30,04:34:30,0 days,1447.00,281.50
4,18.435000,7.413056,-1.071389,4.460833,2015-10-14 23:42:15,2015-10-15 05:24:15,05:42:00,0 days,1422.25,324.25
5,18.499167,7.365556,-1.023889,4.084167,2015-10-12 23:00:15,2015-10-13 07:22:00,08:21:45,0 days,1380.25,442.00
6,18.247778,7.557778,1.029722,-2.618611,NaT,NaT,NaT,NaT,NaN,NaN
7,18.278333,7.533611,-1.450278,3.480000,2015-10-19 22:27:45,2015-10-20 07:28:15,09:00:30,0 days,1347.75,448.25
8,18.696111,7.224722,7.987778,8.949722,2015-10-06 23:02:45,2015-10-07 03:13:15,04:10:30,0 days,1382.75,193.25
